# User Item Biases With Regularization
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = u_i + a_j$
* Loss function is $L = \sum_{\Omega}w_{ij}\text{loss}(r_{ij}, \tilde r_{ij}) + \lambda_u \sum_i (u_i - \bar u) ^2 + \lambda_a \sum_j (a_j - \bar a)^2 $
* $\bar u$ is the mean of $u_i$ and $\bar a$ is the mean of $a_j$ 
* $\Omega$ is the set of oberved pairs $(i, j)$
* $r_{ij}$ is the rating for user $i$ and item $j$
* $w_{ij}$ is the weight for the prediction $r_{ij}$ and is modeled as a power-law in the number of items seen by $i$ and users than have seen $j$: $w_{ij} = |j' : (i, j') \in \Omega| ^ {\lambda_{wu}} |i' : (i', j) \in \Omega| ^ {\lambda_{wa}}$
* $\text{loss}$ is mean squared error

In [1]:
const name = "ExplicitUserItemBiases"
const implicit = false;

In [2]:
import NBInclude: @nbinclude
@nbinclude("Alpha.ipynb");
@nbinclude("ExplicitUserItemBiasesBase.ipynb");

In [3]:
const training = get_split("training", implicit)
const validation = get_split("validation", implicit);

## Alternating Least Squares
* Given some hyperparameters $\lambda$, we can solve for $U$ and $A$ via Alternating Least Squares
* This is an iterative algorithm where we fix $A$, then solve for the $U$ that minimizes the loss function
* Then we fix $U$ and solve for the best $A$
* These two steps are repeated until the matrices $U$ and $A$ converge
### More details
* If we fix $a$, then for each user $i$, $u_i$ is optimized when
* $u_i = \dfrac{\sum_{j \in \Omega_i}(r_{ij} - a_j) w_{ij} + \bar u \lambda_u}{ \sum_{j \in \Omega_i} w_{ij} + \lambda_u}$
* $\Omega$ is the set of (user, item) pairs that we have ratings for
* $\Omega_i$ is subset of $\Omega$ for which the user is the $i$-th user

In [4]:
function train_model(training, stop_criteria, λ)
    @info "training model with parameters $λ"
    λ_u, λ_a, λ_wu, λ_wa = λ
    users, items, ratings = training.user, training.item, training.rating
    weights =
        expdecay(get_counts("training", implicit), log(λ_wu)) .*
        expdecay(get_counts("training", implicit; by_item = true), log(λ_wa))
    u = zeros(eltype(λ_u), num_users())
    a = zeros(eltype(λ_a), num_items())

    ρ_u = zeros(eltype(u), length(u), Threads.nthreads())
    Ω_u = zeros(eltype(u), length(u), Threads.nthreads())
    ρ_a = zeros(eltype(a), length(a), Threads.nthreads())
    Ω_a = zeros(eltype(a), length(a), Threads.nthreads())

    while !stop!(stop_criteria, [u, a])
        update_users!(users, items, ratings, weights, u, a, λ_u, ρ_u, Ω_u)
        update_users!(items, users, ratings, weights, a, u, λ_a, ρ_a, Ω_a)
    end
    u, a
end;

In [5]:
function validation_mse(λ)
    λ = exp.(λ) # ensure λ is nonnegative
    stop_criteria = convergence_stopper(1e-6, max_iters = 16)
    u, a = train_model(training, stop_criteria, λ)
    r = make_prediction(validation.user, validation.item, u, a)
    residualized_loss([], implicit, r)
end;

In [6]:
# Find the best regularization hyperparameters
res = Optim.optimize(
    validation_mse,
    fill(0.0f0, 4),
    Optim.NewtonTrustRegion(),
    autodiff = :forward,
    Optim.Options(show_trace = true, extended_trace = true),
);
λ = exp.(Optim.minimizer(res));

[ Info: 20220625 19:57:37 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,1.0,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,1.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,1.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,0.0,1.0)]
Progress: 100%|███████████████████████████| Time: 0:00:00 (18.34 ns/it)39m
Progress: 100%|███████████████████████████| Time: 0:00:00 ( 4.16 ns/it)
Progress: 100%|███████████████████████████| Time: 0:00:00 (33.27 ns/it)
[ Info: 20220625 19:57:58 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,1.0,0.0,0.0,0.0)

Iter     Function value   Gradient norm 
     0     1.778538e+00     7.672866e-02
 * time: 0.007397890090942383
 * g(x): Float32[-0.0065773665, -1.8496082f-6, 0.019701673, 0.07672866]
 * reached_subproblem_solution: true
 * h(x): Float32[0.0055676643 -2.1045787f-7 -0.0015772729 -0.06168165; -2.1045791f-7 -8.33904f-7 1.2805032f-5 1.1528118f-5; -0.0015772716 1.2805035f-5 0.0027728304 0.0076725623; -0.06168165 1.1528117f-5 0.007672585 0.9672913]
 * x: Float32[0.0, 0.0, 0.0, 0.0]
 * lambda: NaN
 * interior: true
 * hard case: false
 * delta: 1.0


[ Info: 20220625 19:59:14 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0378308,1.0378308,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0009433,0.0,1.0009433,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.36824173,0.0,0.0,0.36824173,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9343703,0.0,0.0,0.0,0.9343703)]


     1     1.778538e+00     7.672866e-02
 * time: 15.757678031921387
 * g(x): Float32[-0.0065773665, -1.8496082f-6, 0.019701673, 0.07672866]
 * reached_subproblem_solution: false
 * h(x): Float32[0.0055676643 -2.1045787f-7 -0.0015772729 -0.06168165; -2.1045791f-7 -8.33904f-7 1.2805032f-5 1.1528118f-5; -0.0015772716 1.2805035f-5 0.0027728304 0.0076725623; -0.06168165 1.1528117f-5 0.007672585 0.9672913]
 * x: Float32[0.0, 0.0, 0.0, 0.0]
 * lambda: 0.016406853
 * interior: false
 * hard case: false
 * delta: 0.25050202


[ Info: 20220625 19:59:29 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0228587,1.0228587,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000753,0.0,1.0000753,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.78556234,0.0,0.0,0.78556234,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.93201923,0.0,0.0,0.0,0.93201923)]
[ Info: 20220625 19:59:44 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0228587,1.0228587,0.0,0.0,0.0),Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0228587,1.0228587,0.0,0.0,0.0),Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.0,0.0,0.0

...

    10     1.772694e+00     1.327470e-07
 * time: 753.5726640224457
 * g(x): Float32[3.4234148f-8, 5.3027454f-11, -1.3274696f-7, 1.0247344f-7]
 * reached_subproblem_solution: true
 * h(x): Float32[0.021567563 -2.261245f-6 -0.04807131 -0.24730137; -2.2612455f-6 1.8499382f-5 -9.916365f-5 -7.9320846f-5; -0.04807131 -9.916365f-5 0.16851261 0.53928; -0.24730138 -7.932085f-5 0.53928 3.098172]
 * x: Float32[-0.24162872, -0.09950643, -0.38635594, -0.02358126]
 * lambda: 0.0
 * interior: true
 * hard case: false
 * delta: 0.50100404


[ Info: 20220625 20:11:47 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.7853306,0.7853306,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9052745,0.0,0.9052745,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.67952895,0.0,0.0,0.67952895,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9766928,0.0,0.0,0.0,0.9766928)]
[ Info: 20220625 20:12:02 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.7853306,0.7853306,0.0,0.0,0.0),Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.7853306,0.7853306,0.0,0.0,0.0),Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.0,0.0,0.0,0

    11     1.772694e+00     1.123424e-06
 * time: 836.1069769859314
 * g(x): Float32[-9.862168f-9, -1.4078085f-11, -1.0403517f-7, 1.1234237f-6]
 * reached_subproblem_solution: true
 * h(x): Float32[0.021567512 -2.2612455f-6 -0.048071153 -0.24730042; -2.2612448f-6 1.8499253f-5 -9.916307f-5 -7.9320154f-5; -0.048071153 -9.916309f-5 0.16851625 0.53927857; -0.24730042 -7.932014f-5 0.53927845 3.0981455]
 * x: Float32[-0.24165049, -0.09951706, -0.3863554, -0.023583125]
 * lambda: 0.0
 * interior: true
 * hard case: false
 * delta: 0.50100404


[ Info: 20220625 20:13:09 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.78529906,0.78529906,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9052623,0.0,0.9052623,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.67952985,0.0,0.0,0.67952985,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9766891,0.0,0.0,0.0,0.9766891)]
[ Info: 20220625 20:13:24 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.78529906,0.78529906,0.0,0.0,0.0),Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.78529906,0.78529906,0.0,0.0,0.0),Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.0,0.0

    12     1.772694e+00     4.080317e-07
 * time: 918.4016778469086
 * g(x): Float32[-1.7941112f-8, -6.264748f-11, 4.0803172f-7, -7.340126f-8]
 * reached_subproblem_solution: true
 * h(x): Float32[0.021567527 -2.261239f-6 -0.04807097 -0.2473006; -2.2612378f-6 1.8499133f-5 -9.916272f-5 -7.932059f-5; -0.04807097 -9.916272f-5 0.16851087 0.5392814; -0.24730058 -7.932058f-5 0.53928125 3.0981536]
 * x: Float32[-0.24169067, -0.09953056, -0.38635412, -0.023586921]
 * lambda: 0.0
 * interior: true
 * hard case: false
 * delta: 0.50100404


In [7]:
@info "The optimal λ is $λ, found in " * repr(Optim.f_calls(res)) * " function calls"

[ Info: 20220625 20:14:32 The optimal λ is Float32[0.78529906, 0.9052623, 0.67952985, 0.9766891], found in 13 function calls


In [8]:
stop_criteria = convergence_stopper(1e-6, max_iters = 16)
u, a = train_model(training, stop_criteria, λ);

[ Info: 20220625 20:14:32 training model with parameters Float32[0.78529906, 0.9052623, 0.67952985, 0.9766891]


In [9]:
validation_mse(Optim.minimizer(res))

[ Info: 20220625 20:14:39 training model with parameters Float32[0.78529906, 0.9052623, 0.67952985, 0.9766891]


1.7725984f0

## Inference

In [10]:
model(users, items) = make_prediction(users, items, u, a)
write_alpha(model, [], implicit, name);

[ Info: 20220625 20:14:48 validation loss: 1.7725984, β: Float32[1.0016403]
[ Info: 20220625 20:14:50 training loss: 1.6141425, β: Float32[1.0016403]


In [11]:
write_params(Dict("u" => u, "a" => a, "λ" => λ), name);